In [20]:
α = [-1, 2]
θ = [2, 1]
# γ = [0, 0]
# β = [0, -0]
γ = [1, 1]
β = [1, -0.5]

include("model.ji")

main (generic function with 2 methods)

In [22]:
n = 200
p = length(α)
@time main(γ, θ, α, β, n)

  2.226173 seconds (35.43 M allocations: 3.707 GiB, 24.03% gc time)


([0.5994039277642657, -0.6865531752578701], [1.7722357020990764 1.6826580193104985])

In [23]:
Theta = zeros(100, p)
Alpha = zeros(100, p)
import Base.Threads.@threads

In [12]:
t = time()
len = 50
@threads for i in 1:2
        for j in 1:len
    Alpha[len*(i - 1) + j, :], Theta[len*(i - 1) + j, :] = main(γ, θ, α, β, n)
    end
end
println("MC Computing time: ", time() - t)
println(sum(Alpha, dims = 1))
println(sum(Theta, dims = 1))

MC Computing time: 184.00889897346497
[-3.232658847877385 -1.5290780293284891]
[42.39233377167564 94.99013746457358]


In [13]:
ct, dc, mc = qgplsim.collection(sign.(Alpha))

(Any[[1.0, -1.0], [1.0, 1.0], [-1.0, 1.0], [-1.0, -1.0]], Dict{Any, Any}([-1.0, -1.0] => [23, 34, 36, 40, 42, 46, 49, 54, 56, 59, 60, 63, 67, 70, 73, 87, 91, 94, 96, 99], [1.0, -1.0] => [1, 3, 11, 20, 21, 22, 25, 29, 30, 31  …  64, 68, 71, 72, 75, 80, 83, 88, 92, 93], [-1.0, 1.0] => [8, 9, 10, 12, 13, 15, 17, 18, 19, 26  …  77, 79, 81, 82, 84, 86, 89, 95, 98, 100], [1.0, 1.0] => [2, 4, 5, 6, 7, 14, 16, 24, 41, 57, 61, 78, 85, 90, 97]), [-1.0, 1.0])

In [14]:
mc

2-element Vector{Float64}:
 -1.0
  1.0

In [17]:
rAlpha = Alpha[dc[mc], :]
rTheta = Theta[dc[mc], :]
ns, = size(rAlpha)

(34, 2)

In [19]:
sum(rAlpha, dims = 1)/ns

1×2 Matrix{Float64}:
 -0.642502  0.637289

In [18]:
sum(rTheta, dims = 1)/ns

1×2 Matrix{Float64}:
 0.860823  1.07853